In [1]:
import torch
from scipy import optimize
import torch.nn.functional as F
import math
import numpy as np
from functools import reduce
from collections import OrderedDict

class PyTorchObjective(object):
    """PyTorch objective function, wrapped to be called by scipy.optimize."""
    def __init__(self, agent):
        self.f = agent.nn # some pytorch module, that produces a scalar loss
        # make an x0 from the parameters in this module
        parameters = OrderedDict(agent.nn.named_parameters())
        self.param_shapes = {n:parameters[n].size() for n in parameters}
        # ravel and concatenate all parameters to make x0
        self.x0 = np.concatenate([parameters[n].data.numpy().ravel() 
                                   for n in parameters])
        
        self.eval_fn = agent.evaluate
        self.c = 0

    def unpack_parameters(self, x):
        """optimize.minimize will supply 1D array, chop it up for each parameter."""
        i = 0
        named_parameters = OrderedDict()
        for n in self.param_shapes:
            param_len = reduce(lambda x,y: x*y, self.param_shapes[n])
            # slice out a section of this length
            param = x[i:i+param_len]
            # reshape according to this size, and cast to torch
            param = param.reshape(*self.param_shapes[n])
            named_parameters[n] = torch.from_numpy(param)
            # update index
            i += param_len
        return named_parameters

    def pack_grads(self):
        """pack all the gradients from the parameters in the module into a
        numpy array."""
        grads = []
        for p in self.f.parameters():
            grad = p.grad.data.numpy()
            grads.append(grad.ravel())
        return np.concatenate(grads)

    def is_new(self, x):
        # if this is the first thing we've seen
        if not hasattr(self, 'cached_x'):
            return True
        else:
            # compare x to cached_x to determine if we've been given a new input
            x, self.cached_x = np.array(x), np.array(self.cached_x)
            error = np.abs(x - self.cached_x)
            return error.max() > 1e-8

    def cache(self, x):
        # unpack x and load into module 
        state_dict = self.unpack_parameters(x)
        self.f.load_state_dict(state_dict)
        # store the raw array as well
        self.cached_x = x
        # zero the gradient
        self.f.zero_grad()
        # use it to calculate the objective
        obj = self.eval_fn()
        # backprop the objective
        # obj.backward()
        self.cached_f = obj
        return obj

    def fun(self, x):
        self.c += 1
        if self.is_new(x):
            # print(self.c)
            self.cache(x)
        return self.cached_f

In [2]:
import gym_gvgai

In [3]:
from agent.NNagent import NNagent

In [4]:
from generator.env_gen_wrapper import GridGame

In [5]:
from scipy.optimize import Bounds

In [6]:
_x = NNagent(GridGame(game='zelda', 
                     play_length=200,
                     path=gym_gvgai.dir + '/envs/games/zelda_v0/', 
                     lvl_name='zelda_lvl0.txt', 
                     mechanics=['1', '2', '3', '+', 'g', 'w'], # monsters, key, door, wall
                  )
         )

Connecting to host 127.0.0.1 at port 52831 ...
Client connected to server [OK]


In [7]:
_x

In [8]:
_x.nn

Net(
  (conv1): Conv2d(13, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=96, out_features=48, bias=True)
  (fc2): Linear(in_features=48, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=6, bias=True)
)

In [9]:
z = PyTorchObjective(_x)

In [10]:
z.x0.shape

(9262,)

In [11]:
tile = _x.reset()

In [12]:
o = _x.get_action(tile)

In [13]:
o

5

In [14]:
bounds = [(-1, 1)]*z.x0.shape[0]

In [21]:
pop = np.random.randn(99, z.x0.shape[0])

In [22]:
pop.shape

(99, 9262)

In [23]:
q = np.vstack((z.x0, pop))

In [25]:
from utils.diff_evo import differential_evolution

In [ ]:
import time
start = time.time()

In [26]:
ans = differential_evolution(z.fun, bounds, popsize=q.shape[0], polish=False, init=q)

100


In [27]:
end = time.time() - start

NameError: name 'time' is not defined

In [ ]:
end

In [ ]:
ans.x

In [ ]:
z.fun(ans.x)